In [ ]:
import sys
import os
from pathlib import Path

# --- ROBUST PATH SETUP ---
# 1. Get the directory of this script (e.g., .../fomc)
current_script_dir = Path(__file__).resolve().parent
# 2. Get the project root (e.g., .../trading_analyst_agent)
project_root = current_script_dir.parent

# 3. Add project root to sys.path so we can import 'config'
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
# -------------------------

import json
from datetime import datetime

# Now we can safely import from the project
from config import get_config
from fomc.fomc_workflow import FOMCAnalysisWorkflow
from fomc.fomc_database import create_database_schema
from fomc.fomc_telegram import send_fomc_notification

def main():
    """
    Run the FOMC Analysis Workflow
    """
    # Load configuration
    config = get_config()
    
    # Create database schema (ensure tables exist)
    print("Creating database schema...")
    create_database_schema(config["db_connection_params"])
    
    # Initialize the workflow
    print("\nInitializing FOMC Analysis Workflow...")
    workflow = FOMCAnalysisWorkflow(
        anthropic_api_key=config["anthropic_api_key"],
        db_connection_params=config["db_connection_params"],
        langsmith_api_key=config.get("langsmith_api_key")
    )
    
    # Run the workflow (Auto-detects dates from URL)
    print(f"\n{'='*80}")
    print(f"Running FOMC Analysis Pipeline")
    print(f"{'='*80}\n")
    
    target_date = sys.argv[1] if len(sys.argv) > 1 else None
    
    if target_date:
        print(f"Force Mode Enabled: Targeting {target_date}")
    
    result = workflow.run(force_date=target_date)
    
    print("\n" + "="*50)
    print("EXECUTION REPORT")
    print("="*50)
    
    print(f"Status: {result.get('status')}")
    print(f"Message: {result.get('message')}")
    
    if result.get('status') == 'success' and result.get('data'):
         synthesis = result['data'].get('meeting_cycle_synthesis', {})
         regime = synthesis.get('policy_regime', {}).get('current', 'Unknown')
         print(f"New Analysis Generated. Regime: {regime}")
         
         # Send Telegram notifications if bot token is configured
         if config.get("telegram_bot_token"):
             try:
                 telegram_result = send_fomc_notification(
                     analysis_data=result['data'],
                     telegram_bot_token=config["telegram_bot_token"],
                     openai_api_key=config["openai_api_key"],
                     db_connection_params=config["db_connection_params"]
                 )
                 
                 print(f"\n📱 Telegram Notifications: {telegram_result['success']} sent, {telegram_result['failed']} failed")
                 
             except Exception as e:
                 print(f"\n⚠️ Telegram notification failed: {e}")
         else:
             print("\nℹ️ Telegram bot token not configured. Skipping notifications.")
             print("   Set TELEGRAM_BOT_TOKEN environment variable or add to config.py")


if __name__ == "__main__":
    main()